In [1]:
from scrapper import AudibleScrapper
from data_manager import DataManager
from book_graph import BookGraph
from link_dataset import LinkDataset
from link_predictor import LinkPredictor

import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torch_geometric.transforms import RandomLinkSplit

%reload_ext autoreload
%autoreload 2

Global seed set to 3088024102


Dump data from database

In [2]:
dm = DataManager()
dump = dm.get_connected_scrapped_subgraph()

Create a graph from the dump and convert it to usable tensors.

In [3]:
book_graph = BookGraph(dump)
data = book_graph.to_torch()
book_graph.plot(sample=10, figsize=(20,10))

Split the data into three sets (Train / Test / Val)

In [ ]:
transform = RandomLinkSplit()
train_data, val_data, test_data = transform(data)

Create Dataset objects for each of these sets

In [ ]:
train_dataset = LinkDataset(train_data, data)
val_dataset = LinkDataset(val_data, data)
test_dataset = LinkDataset(test_data, data)

Create loader objects for each sets

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=1)
val_loader = DataLoader(val_dataset, batch_size=1)
test_loader = DataLoader(test_dataset, batch_size=1)

Create model for predicting links

In [ ]:
# Create model for predicting links.
model = LinkPredictor(
        in_channels=4,
        hidden_channels=16,
        out_channels=1,
        embedding_size=16,
        num_nodes=data.num_nodes
        )

In [ ]:
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, train_loader, val_loader)